In [12]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler


In [2]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [14]:
scaler = MinMaxScaler()

In [3]:
df = df.drop(["Surname", "CustomerId", "RowNumber"], axis = 1)
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [4]:
le = LabelEncoder()

In [5]:
df["Geography"] = le.fit_transform(df["Geography"])
df["Gender"] = le.fit_transform(df["Gender"])
df


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0,0,42,2,0.00,1,1,1,101348.88,1
1,608,2,0,41,1,83807.86,1,0,1,112542.58,0
2,502,0,0,42,8,159660.80,3,1,0,113931.57,1
3,699,0,0,39,1,0.00,2,0,0,93826.63,0
4,850,2,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,0,1,39,5,0.00,2,1,0,96270.64,0
9996,516,0,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,0,0,36,7,0.00,1,0,1,42085.58,1
9998,772,1,1,42,3,75075.31,2,1,0,92888.52,1


In [16]:
X = df.loc[:, df.columns != 'Exited'].to_numpy()
y = df["Exited"].to_numpy()

In [18]:
X = scaler.fit_transform(X)

In [23]:
X

array([[0.538     , 0.        , 0.        , ..., 1.        , 1.        ,
        0.50673489],
       [0.516     , 1.        , 0.        , ..., 0.        , 1.        ,
        0.56270874],
       [0.304     , 0.        , 0.        , ..., 1.        , 0.        ,
        0.56965435],
       ...,
       [0.718     , 0.        , 0.        , ..., 0.        , 1.        ,
        0.21039009],
       [0.844     , 0.5       , 1.        , ..., 1.        , 0.        ,
        0.46442905],
       [0.884     , 0.        , 0.        , ..., 1.        , 0.        ,
        0.19091423]])

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [30]:
class ANN_classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_stack = nn.Sequential(
            nn.Linear(10,24),
            nn.ReLU(),
            nn.Linear(24,48),
            nn.ReLU(),
            nn.Linear(48,12),
            nn.ReLU(),
            nn.Linear(12,1),
        )

    def forward(self,x):
        x = self.linear_stack(x)
        x = torch.sigmoid(x)
        return x

In [31]:
model = ANN_classifier()
print(model)

ANN_classifier(
  (linear_stack): Sequential(
    (0): Linear(in_features=10, out_features=24, bias=True)
    (1): ReLU()
    (2): Linear(in_features=24, out_features=48, bias=True)
    (3): ReLU()
    (4): Linear(in_features=48, out_features=12, bias=True)
    (5): ReLU()
    (6): Linear(in_features=12, out_features=1, bias=True)
  )
)


In [32]:
model = ANN_classifier()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
inputs = torch.tensor(X_train, dtype=torch.float32)
labels = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)

epochs = 275
for epoch in range(epochs):

    outputs = model(inputs)
    loss = criterion(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [10/275], Loss: 0.7475
Epoch [20/275], Loss: 0.7013
Epoch [30/275], Loss: 0.6481
Epoch [40/275], Loss: 0.5851
Epoch [50/275], Loss: 0.5236
Epoch [60/275], Loss: 0.4986
Epoch [70/275], Loss: 0.5001
Epoch [80/275], Loss: 0.4958
Epoch [90/275], Loss: 0.4937
Epoch [100/275], Loss: 0.4917
Epoch [110/275], Loss: 0.4888
Epoch [120/275], Loss: 0.4858
Epoch [130/275], Loss: 0.4824
Epoch [140/275], Loss: 0.4786
Epoch [150/275], Loss: 0.4746
Epoch [160/275], Loss: 0.4699
Epoch [170/275], Loss: 0.4648
Epoch [180/275], Loss: 0.4594
Epoch [190/275], Loss: 0.4537
Epoch [200/275], Loss: 0.4475
Epoch [210/275], Loss: 0.4411
Epoch [220/275], Loss: 0.4348
Epoch [230/275], Loss: 0.4291
Epoch [240/275], Loss: 0.4234
Epoch [250/275], Loss: 0.4178
Epoch [260/275], Loss: 0.4120
Epoch [270/275], Loss: 0.4057


In [33]:
y_pred = model(torch.tensor(X_test, dtype=torch.float32))

In [34]:
y_pred

tensor([[0.2041],
        [0.3529],
        [0.1620],
        ...,
        [0.2746],
        [0.1095],
        [0.1793]], grad_fn=<SigmoidBackward0>)

In [35]:
with torch.no_grad():
    y_predicted = model(torch.tensor(X_test, dtype=torch.float32))
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(torch.tensor(y_test).unsqueeze(1)).sum() / float(y_test.shape[0])
    print(f'Accuracy: {acc:.4f}')

Accuracy: 0.8315
